# Initial EDA for Opioid Abuse Topic

In [ ]:
import sas7bdat as sas
import sqlite3
import time

The medicare data is a couple of gigs, so load it into a db rather than into memory

In [3]:

i = 0
for line in sas.SAS7BDAT("./pp_ccwid_bn_2012_r.sas7bdat"):
    print line
    i += 1
    if i > 5:
        break

[u'UNIQUE_PRSCRBR_ID', u'BN', u'BENE_COUNT', u'CLAIM_COUNT', u'CLAIM_COUNT_DAW', u'CLAIM_COUNT_CMPND', u'QUANTITY_SUM', u'DAY_SUPPLY_SUM', u'GROSS_DRUG_COST_SUM1']
[1.0, u'ACETAMINOPHEN-CODEINE', -6.0, 14.0, 0.0, 0.0, 1130.0, 387.0, 275.46000000000004]
[1.0, u'ACYCLOVIR', -6.0, 29.0, 0.0, 0.0, 2106.0, 510.0, 629.94]
[1.0, u'ADVAIR DISKUS', -6.0, 24.0, 0.0, 0.0, 1440.0, 720.0, 6035.460000000001]
[1.0, u'AGGRENOX', -6.0, 11.0, 0.0, 0.0, 660.0, 330.0, 2616.1]
[1.0, u'ALENDRONATE SODIUM', -6.0, 56.0, 0.0, 0.0, 382.0, 2689.0, 650.47]


In [4]:
raw_data = sas.SAS7BDAT("./pp_ccwid_bn_2012_r.sas7bdat")

Make a small sample set for testing

In [5]:
sample_data = []

i = 0
for line in raw_data:
    if i > 0:
        sample_data.append(line)
    if i > 5:
        break
    i += 1

        

In [6]:
for line in sample_data:
    print line

[1.0, u'ACETAMINOPHEN-CODEINE', -6.0, 14.0, 0.0, 0.0, 1130.0, 387.0, 275.46000000000004]
[1.0, u'ACYCLOVIR', -6.0, 29.0, 0.0, 0.0, 2106.0, 510.0, 629.94]
[1.0, u'ADVAIR DISKUS', -6.0, 24.0, 0.0, 0.0, 1440.0, 720.0, 6035.460000000001]
[1.0, u'AGGRENOX', -6.0, 11.0, 0.0, 0.0, 660.0, 330.0, 2616.1]
[1.0, u'ALENDRONATE SODIUM', -6.0, 56.0, 0.0, 0.0, 382.0, 2689.0, 650.47]
[1.0, u'ALLOPURINOL', -6.0, 35.0, 0.0, 0.0, 1830.0, 1830.0, 216.96000000000004]


In [8]:
conn = sqlite3.connect("drugs.db")

In [10]:
conn.execute\
('CREATE TABLE medd (perscriber_id int, bn text, bene_count int, claim_count int, clain_count_daw int,' + \
 'claim_count_cmpnd int, quantity_sum int, day_supply_sum int, gross_drug_cost_sum real);')

OperationalError: table medd already exists

In [11]:
conn.executemany("INSERT INTO medd VALUES (?,?,?,?,?,?,?,?,?);", sample_data)

In [12]:
for row in conn.execute("SELECT * FROM medd;"):
    print row

(1, u'ACETAMINOPHEN-CODEINE', -6, 14, 0, 0, 1130, 387, 275.46000000000004)
(1, u'ACYCLOVIR', -6, 29, 0, 0, 2106, 510, 629.94)
(1, u'ADVAIR DISKUS', -6, 24, 0, 0, 1440, 720, 6035.460000000001)
(1, u'AGGRENOX', -6, 11, 0, 0, 660, 330, 2616.1)
(1, u'ALENDRONATE SODIUM', -6, 56, 0, 0, 382, 2689, 650.47)
(1, u'ALLOPURINOL', -6, 35, 0, 0, 1830, 1830, 216.96000000000004)


In [19]:
conn.execute("DELETE FROM medd;")

In [20]:
for row in conn.execute("SELECT * FROM medd;"):
    print row

Load the actual data.  ~ 45 minutes on a nano EC2 instance.

In [22]:
start_time = time.time()
conn.executemany("INSERT INTO medd VALUES (?,?,?,?,?,?,?,?,?);", raw_data)
duration = time.time() - start_time
print "Finished in " + str(duration) + "s"

Finished in 2764.21442914s


Look for some basic opioids.  There are a lot.

In [29]:
for row in conn.execute("SELECT bn, COUNT(bn) FROM medd GROUP BY bn HAVING bn LIKE '%OXYCODONE%';"):
    print row

(u'OXYCODONE CONCENTRATE', 473)
(u'OXYCODONE HCL', 74103)
(u'OXYCODONE HCL-ACETAMINOPHEN', 45724)
(u'OXYCODONE HCL-ASPIRIN', 204)
(u'OXYCODONE HCL-IBUPROFEN', 5)
(u'OXYCODONE-ACETAMINOPHEN', 120276)
(u'OXYCODONE-ASPIRIN', 10)


Check how many unique perscribers there are.  ~783k

In [30]:
for row in conn.execute("SELECT COUNT(DISTINCT perscriber_id) FROM medd;"):
    print row

(783117,)


The perscriber IDs are normalized, though, and won't help in joining to other data.

In [32]:
i = 0
for row in conn.execute("SELECT DISTINCT perscriber_id FROM medd;"):
    print row
    i += 1
    if i > 50:
        break

(u'UNIQUE_PRSCRBR_ID',)
(1,)
(2,)
(3,)
(5,)
(6,)
(7,)
(9,)
(10,)
(13,)
(15,)
(17,)
(18,)
(19,)
(21,)
(23,)
(25,)
(27,)
(30,)
(31,)
(32,)
(33,)
(34,)
(35,)
(36,)
(39,)
(41,)
(42,)
(44,)
(46,)
(47,)
(50,)
(54,)
(55,)
(58,)
(59,)
(61,)
(62,)
(66,)
(68,)
(69,)
(70,)
(72,)
(75,)
(78,)
(79,)
(80,)
(85,)
(88,)
(89,)
(90,)


There's potential for graph-based analysis of drug perscriptions, but since the opioid prompt seems to be looking for early detection medicare data isn't great.  What about the End Stage Renal Disease prompt?  This seems just right for medicare data.  Check for common perscrptions for Chronic Kidney Disease symptoms.

In [33]:
for row in conn.execute("SELECT bn, COUNT(bn) FROM medd GROUP BY bn HAVING bn LIKE '%CALCITRIOL%';"):
    print row

(u'CALCITRIOL', 19433)


In [34]:
for row in conn.execute("SELECT bn, COUNT(bn) FROM medd GROUP BY bn HAVING bn LIKE '%PARICALCITOL%';"):
    print row

Might be useful for later.

In [35]:
conn.commit()